In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.models import load_model

2024-01-25 01:39:44.335191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-25 01:39:44.397954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
imgs_path = "./Train"
data = []
labels = []
CLASSES = 43
# using for loop to access each image
for i in range(CLASSES):
    img_path = os.path.join(imgs_path, str(i)) #0-42
    for img in os.listdir(img_path):
        im = Image.open(imgs_path + '/' + str(i) + '/' + img)
        im = im.resize((75,75))
        im = np.array(im)
        data.append(im)
        labels.append(i)
data = np.array(data)
labels = np.array(labels)
print("data[0]: ",data[0])
print("labels[0: ]",labels[0])

data[0]:  [[[118  84  73]
  [134  94  86]
  [167 117 114]
  ...
  [205 158 126]
  [243 223 222]
  [255 253 255]]

 [[128  91  84]
  [141  99  94]
  [169 115 118]
  ...
  [205 158 120]
  [243 221 213]
  [255 250 249]]

 [[149 107 106]
  [157 108 113]
  [172 112 129]
  ...
  [206 158 107]
  [244 217 192]
  [255 244 229]]

 ...

 [[140 193 197]
  [128 177 181]
  [101 142 145]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[111 169 164]
  [102 154 147]
  [ 82 123 110]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 98 158 148]
  [ 90 144 131]
  [ 73 114  94]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
labels[0: ] 0


In [3]:
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
print("training shape: ",x_train.shape, y_train.shape)
print("testing shape: ",x_val.shape, y_val.shape)
# convert interge label to one-hot data
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

print(y_train[1])

training shape:  (31367, 75, 75, 3) (31367,)
testing shape:  (7842, 75, 75, 3) (7842,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
from sklearn.metrics import accuracy_score
test = pd.read_csv("./Test.csv")
test_labels = test['ClassId'].values.tolist()

test_img_path = "./"
test_imgs = test['Path'].values
test_data = []

for img in test_imgs:
    im = Image.open(test_img_path + '/' + img)
    im = im.resize((75,75))
    im = np.array(im)
    test_data.append(im)
test_data = np.array(test_data)

In [5]:
from tensorflow.keras import layers, models

# Transfer Learning using Xception

In [6]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input


xception = Xception(weights= 'imagenet', include_top=False, input_shape= (75,75,3))

In [7]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model


x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(43, activation= 'softmax')(x)
model = Model(inputs = xception.input, outputs = predictions)

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

model_check = ModelCheckpoint('convnet_for_GTSRB_with_Xception_ImageNet_weights.keras', monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=0, mode='max', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

csv_logger = CSVLogger('train_log.csv', separator=',')

In [10]:
n_epochs = 50
history =  model.fit(x_train, y_train,  batch_size = 32, epochs = n_epochs, verbose = 1, 
              validation_data = (x_val, y_val), callbacks = [model_check, early, reduce_lr, csv_logger])

Epoch 1/50
981/981 [==============================] - 1731s 2s/step - loss: 0.2670 - accuracy: 0.9304 - val_loss: 0.0583 - val_accuracy: 0.9858 - lr: 0.0010
Epoch 2/50
981/981 [==============================] - 1719s 2s/step - loss: 0.0420 - accuracy: 0.9903 - val_loss: 0.0655 - val_accuracy: 0.9865 - lr: 0.0010
Epoch 3/50
981/981 [==============================] - 1126s 1s/step - loss: 0.0331 - accuracy: 0.9919 - val_loss: 0.0462 - val_accuracy: 0.9871 - lr: 0.0010
Epoch 4/50
981/981 [==============================] - 1001s 1s/step - loss: 0.0264 - accuracy: 0.9944 - val_loss: 0.0175 - val_accuracy: 0.9957 - lr: 0.0010
Epoch 5/50
981/981 [==============================] - 937s 955ms/step - loss: 0.0204 - accuracy: 0.9960 - val_loss: 0.0904 - val_accuracy: 0.9847 - lr: 0.0010
Epoch 6/50
981/981 [==============================] - 976s 995ms/step - loss: 0.0198 - accuracy: 0.9957 - val_loss: 0.0782 - val_accuracy: 0.9861 - lr: 0.0010
Epoch 7/50
981/981 [==============================] - 

In [11]:
model.save('convnet_for_GTSRB_with_Xception_ImageNet_weights.keras')

In [13]:
from tensorflow import keras

test_model = keras.models.load_model("convnet_for_GTSRB_with_Xception_ImageNet_weights.keras")
predictions = test_model.predict(test_data)
classes_x = np.argmax(predictions, axis = 1).tolist()
classes_x = np.array([classes_x]).tolist()[0]

print("Accuracy on test dataset using CNN after transfer learning with Xception: ", accuracy_score(test_labels, classes_x))

395/395 [==============================] - 75s 188ms/step
Accuracy on test dataset using CNN after transfer learning with Xception:  0.9812351543942993
